In [317]:
using Catlab
using Catlab.CategoricalAlgebra
using Catlab.Graphs
using Catlab.Graphs.BasicGraphs
using Catlab.Graphics
using Catlab.Graphics.Graphviz
using Catlab.Graphics.GraphvizGraphs
using Catlab.Graphs.PropertyGraphs
using Catlab.CategoricalAlgebra.CSets
using Catlab.CategoricalAlgebra.FinSets, Catlab.CategoricalAlgebra
using DataStructures
using Colors
using Random

In [318]:
draw(g) = to_graphviz(g, node_labels=true)

draw (generic function with 1 method)

In [319]:
struct Preorder
    carrier:: Set{String}
    relation :: SortedSet{Pair{String,String}}
    """Construct valid preorder by taking reflexive/transitive closure"""
    function Preorder(carrier:: Set{String}, rel :: Vector{Pair{String,String}})
      for (a,b) ∈ rel
        a ∈ carrier && b ∈ carrier || error("relation element not in carrier set")
      end
      relation = SortedSet(rel)
      for (a, b, c) in Iterators.product(carrier, carrier, carrier)
        if ((a => b) ∈ relation) && ((b => c) ∈ relation)
          push!(relation, a => c) # enforces relation is transitive
        end
      end
      for i in carrier
        push!(relation, i => i) # enforces that relation is reflexive
      end
      return new(carrier, relation)
    end
  end

In [332]:
function load_major(path::String)::Pair{Set{String}, Vector{Pair{String, String}}}
    file = open(path, "r")
    classes = Set{String}()
    relations = Vector{Pair{String, String}}()
    while !eof(file)
        line = readline(file)
        if length(strip(line, ' ')) != 0
            sp = split(line, ":")

            if length(sp) >= 1
                name = strip(sp[1], ' ')
                push!(classes, string(name))
            end

            if length(sp) >= 2
                reqs_str = strip(sp[2], ['[', ']', ' '])
                reqs = split(reqs_str, ",")
                
                for req in reqs
                    req = strip(req, ' ')
                    if length(req) > 0
                        if !in(req, classes)
                            push!(classes, req)
                        end
                        push!(relations, req => name)
                    end
                end
            end
        end
    end
    close(file)
    return classes => relations
end

load_major (generic function with 1 method)

In [333]:
cs = load_major("reqs/CS.reqs")

Set(["Sem6", "Humanities1", "Social/Behavioral1", "MAC2312", "CIS4914", "Sem2", "ENC3246", "CNT4007", "Sem8", "Sem4"  …  "HumOrSocOrPhy1", "MAS3114", "STA3032", "COP3502", "CDA3101", "MAC2311", "Sem1", "Sem7", "Social/Behavioral2", "Sem3"]) => ["MAC2311" => "MAC2312", "MAC2312" => "MAC2313", "MAC2312" => "MAS3114", "MAC2311" => "STA3032", "MAC2311" => "PHY2048/PHY2048L", "PHY2048/PHY2048L" => "PHY2049/PHY2049L", "MAC2313" => "PHY2049/PHY2049L", "MAC2311" => "COP3502", "MAC2311" => "COP3503", "COP3502" => "COP3503"  …  "COP3530" => "COP4533", "COP4600" => "CNT4007", "Sem6" => "CIS4914", "Sem7" => "Sem8", "Sem6" => "Sem7", "Sem5" => "Sem6", "Sem4" => "Sem5", "Sem3" => "Sem4", "Sem2" => "Sem3", "Sem1" => "Sem2"]

In [359]:
function top_sort(vertices::Set{String}, edges::Vector{Pair{String,String}}, seed::Int = 0)
    Random.seed!(seed)

    l = []
    q = Queue{String}()
    d = Dict{String, Int}()

    for v in vertices
        d[v] = 0
        for (a,b) in edges
            if v == b && a != b
                d[v]+=1
            end
        end
    end


    
    # randomizes the starting order of the queue
    for v in shuffle!(collect(vertices))
        if d[v] == 0
            enqueue!(q, v)
        end
    end


    while !isempty(q)
        v = dequeue!(q)
        push!(l, v)

        for (a,b) in shuffle!(collect(edges))
            if v == a && a != b
                d[b]-=1
                if d[b] == 0
                    enqueue!(q, b)
                end
            end
        end
    end
    
    return l
end

top_sort (generic function with 4 methods)

In [362]:
for class in top_sort(cs.first, cs.second)
    println(class)
end

Sem1
Social/Behavioral1
MAC2311
Social/Behavioral2
Humanities1
ENC3246
Humanities2
HumOrSocOrPhy1
EGS4034
Sem2
STA3032
PHY2048/PHY2048L
MAC2312
COP3502
Sem3
MAC2313
MAS3114
COP3503
Sem4
PHY2049/PHY2049L
COT3100
Sem5
COP3530
CDA3101
Sem6
COP4020
COP4533
CEN3031
CIS4301
COP4600
Sem7
CIS4914
CNT4007
Sem8


In [339]:
cs.second

34-element Vector{Pair{String, String}}:
          "MAC2311" => "MAC2312"
          "MAC2312" => "MAC2313"
          "MAC2312" => "MAS3114"
          "MAC2311" => "STA3032"
          "MAC2311" => "PHY2048/PHY2048L"
 "PHY2048/PHY2048L" => "PHY2049/PHY2049L"
          "MAC2313" => "PHY2049/PHY2049L"
          "MAC2311" => "COP3502"
          "MAC2311" => "COP3503"
          "COP3502" => "COP3503"
                    ⋮
          "COP4600" => "CNT4007"
             "Sem6" => "CIS4914"
             "Sem7" => "Sem8"
             "Sem6" => "Sem7"
             "Sem5" => "Sem6"
             "Sem4" => "Sem5"
             "Sem3" => "Sem4"
             "Sem2" => "Sem3"
             "Sem1" => "Sem2"